<a href="https://colab.research.google.com/github/eonlabs-research/rsr-fsa/blob/main/sr_mdd_equity_returns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly


In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from copy import deepcopy


In [7]:
def new_sharpe_ratio(returns, annual_rf):
    excess_returns = returns - annual_rf/365
    return np.mean(excess_returns) / np.std(excess_returns, ddof=1) * np.sqrt(365)

def calculate_max_drawdown(nav_values):
    return np.max(1 - nav_values / np.maximum.accumulate(nav_values))

def pso_fitness_function_v2(returns, annual_rf, target_sharpe=4, target_drawdown=0.033):
    sharpe = new_sharpe_ratio(returns, annual_rf)
    max_drawdown = calculate_max_drawdown((returns + 1).cumprod())

    sharpe_penalty = -abs(sharpe - target_sharpe) * 10
    drawdown_penalty = -abs(max_drawdown - target_drawdown) * 10

    return sharpe_penalty + drawdown_penalty


In [8]:
NUM_PARTICLES = 100
MAX_ITERATIONS = 100
INERTIA = 0.5
PERSONAL_BEST_WEIGHT = 1.5
GLOBAL_BEST_WEIGHT = 1.5

class Particle:
    def __init__(self):
        self.position = np.random.normal(0, 0.01, 365)
        self.velocity = np.random.uniform(-0.01, 0.01, 365)
        self.best_position = np.copy(self.position)
        self.best_score = -np.inf

    def update(self, global_best_position):
        inertia_component = INERTIA * self.velocity
        personal_best_component = PERSONAL_BEST_WEIGHT * np.random.random() * (self.best_position - self.position)
        global_best_component = GLOBAL_BEST_WEIGHT * np.random.random() * (global_best_position - self.position)
        self.velocity = inertia_component + personal_best_component + global_best_component
        self.position += self.velocity

def particle_swarm_optimization_v5(annual_rf, target_sharpe=4, target_drawdown=0.033):
    particles = [Particle() for _ in range(NUM_PARTICLES)]
    global_best_position = np.random.normal(0, 0.01, 365)
    global_best_score = -np.inf

    for iteration in range(MAX_ITERATIONS):
        for particle in particles:
            fitness = pso_fitness_function_v2(particle.position, annual_rf, target_sharpe, target_drawdown)
            if fitness > particle.best_score:
                particle.best_score = fitness
                particle.best_position = deepcopy(particle.position)

            if fitness > global_best_score:
                global_best_score = fitness
                global_best_position = deepcopy(particle.position)

        for particle in particles:
            particle.update(global_best_position)

    return global_best_position, (global_best_position + 1).cumprod()


In [11]:
risk_free_rate = 0.01  # Example annual risk-free rate

NUM_RUNS = 10  # Number of optimization runs

strict_results_final = []

for _ in range(NUM_RUNS):
    returns_strict, equity_curve_strict = particle_swarm_optimization_v5(risk_free_rate)
    sharpe_strict = new_sharpe_ratio(returns_strict, risk_free_rate)
    mdd_strict = calculate_max_drawdown(equity_curve_strict)
    if (3.9 <= sharpe_strict <= 4.1) and (0.031 <= mdd_strict <= 0.035):
        strict_results_final.append((returns_strict, equity_curve_strict, sharpe_strict, mdd_strict))

fig_strict_final = go.Figure()

for i, (returns, equity_curve, sharpe, mdd) in enumerate(strict_results_final):
    fig_strict_final.add_trace(go.Scatter(y=equity_curve, mode='lines', name=f"Run {i+1}: Avg Daily={np.mean(returns):.7f}, Cumulative={equity_curve[-1]-1:.7f}, Sharpe={sharpe:.7f}, MDD={mdd:.4f}"))

fig_strict_final.update_layout(
    title="Equity Curves with Final Sharpe Ratio Calculation",
    xaxis_title="Days",
    yaxis_title="Equity Value",
    legend_title="Runs",
    font=dict(size=10),
    hovermode="x unified",
    legend=dict(y=-0.2, x=0.5, xanchor='center', yanchor='top')  # This line adjusts the legend's position.
)

fig_strict_final.show()
